In [1]:
from datetime import datetime
from functools import partial
import glob
import gc
from multiprocessing import Pool

# Disable multiprocesing for numpy/opencv. We already multiprocess ourselves, this would mean every subprocess produces
# even more threads which would lead to a lot of context switching, slowing things down a lot.
import os
os.environ["OMP_NUM_THREADS"] = "1"

import matplotlib.pyplot as plt
import random
import time
import pandas as pd
import cv2
from PIL import Image
import numpy as np
from tqdm import tqdm, tqdm_notebook
import scipy
import scipy.ndimage
import scipy.special
from scipy.spatial.transform import Rotation as R

import torch
from torch.utils.data import TensorDataset, DataLoader,Dataset
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.optim.optimizer import Optimizer
import torch.backends.cudnn as cudnn
from torch.autograd import Variable
from torch.utils.data.sampler import SubsetRandomSampler
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau, CosineAnnealingLR, _LRScheduler
import torch.nn.utils.weight_norm as weightNorm
import torch.nn.init as init
from torch.nn.parallel.data_parallel import data_parallel

from unet_baseline.models.model import *
from deeplab_v3_baseline.semantic_segmentation.network.deepv3 import *
from deeplab_v3_baseline.tuils.loss_function import *
import torchvision.models as models

from apex import amp

import albumentations
from albumentations import torch as AT

from lyft_dataset_sdk.lyftdataset import LyftDataset
from lyft_dataset_sdk.eval.detection.mAP_evaluation import Box3D, recall_precision
from lyft_dataset_sdk.utils.data_classes import LidarPointCloud, Box, Quaternion
from lyft_dataset_sdk.utils.geometry_utils import view_points, transform_matrix
from lyft_dataset_sdk.utils.map_mask import MapMask
from pathlib import Path
from lyft_dataset_sdk.lyftdataset import LyftDataset,LyftDatasetExplorer

In [4]:
train_data_folder = '/media/jionie/my_disk/Kaggle/Lyft/input/3d-object-detection-for-autonomous-vehicles/train_root/bev_data_with_map'

level5data_train = LyftDataset(data_path='/media/jionie/my_disk/Kaggle/Lyft/input/3d-object-detection-for-autonomous-vehicles/train_root', \
    json_path='/media/jionie/my_disk/Kaggle/Lyft/input/3d-object-detection-for-autonomous-vehicles/train_root/data', verbose=True)
classes = ["car", "motorcycle", "bus", "bicycle", "truck", "pedestrian", "other_vehicle", "animal", "emergency_vehicle"]

9 category,
18 attribute,
4 visibility,
18421 instance,
10 sensor,
148 calibrated_sensor,
177789 ego_pose,
180 log,
180 scene,
22680 sample,
189504 sample_data,
638179 sample_annotation,
1 map,
Done loading in 6.5 seconds.
Reverse indexing ...
Done reverse indexing in 2.3 seconds.


In [5]:
train = pd.read_csv("/media/jionie/my_disk/Kaggle/Lyft/input/3d-object-detection-for-autonomous-vehicles/train_root/train.csv")
sample_tokens = train.Id

In [29]:
wlh_list = []

In [34]:
for i in range(len(sample_tokens)):
    sample = level5data_train.get("sample", sample_tokens[i]) 
    sample_lidar_token = sample["data"]["LIDAR_TOP"]
    boxes = level5data_train.get_boxes(sample_lidar_token)
    
    for j in range(len(boxes)):
        box = boxes[j]
        wlh = box.wlh
        wlh_list.append({'name' : box.name, 'width' : wlh[0], 'length' : wlh[1], 'height' : wlh[2]})

wlh_dataframe = pd.DataFrame(wlh_list)
del wlh_list
gc.collect()

3770673

In [35]:
wlh_dataframe.shape

(639201, 4)

In [36]:
wlh_dataframe.to_csv("wlh_info.csv")